In [1]:
import numpy as np
import tnn
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as f

from datasets import load_dataset

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

In [2]:
dataset = load_dataset("ylecun/mnist", num_proc=2)
train_size = 60000
test_size = 10000

train = dataset.get("train")
test = dataset.get("test")

train_indices = np.random.choice(len(train), size=train_size, replace=False)
test_indices = np.random.choice(len(test), size=test_size, replace=False)

train = train.select(train_indices)
test = test.select(test_indices)

In [3]:
def to_numpy(example):
    arr = np.reshape(example["image"], -1) / 255.0
    example["input"] = arr
    return example


train_dataset = train.map(to_numpy, num_proc=2).select_columns(["input", "label"])
test_dataset = test.map(to_numpy, num_proc=2).select_columns(["input", "label"])

Map (num_proc=2):   0%|          | 0/60000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
def collate_fn(batch):
    inputs = torch.tensor([ex["input"] for ex in batch]).float()
    labels = torch.tensor([ex["label"] for ex in batch]).long()
    return inputs, labels


trainloader = data.DataLoader(
    train_dataset,
    batch_size=len(train_dataset),
    shuffle=True,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=2,
)
testloader = data.DataLoader(
    test_dataset,
    batch_size=len(test_dataset),
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=2,
)

In [5]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(28 * 28, 512)
        self.norm_1 = nn.LayerNorm(512)
        self.drop_1 = nn.Dropout(0.4)
        self.linear_2 = nn.Linear(512, 512)
        self.norm_2 = nn.LayerNorm(512)
        self.drop_2 = nn.Dropout(0.2)
        self.linear_3 = nn.Linear(512, 512)
        self.norm_3 = nn.LayerNorm(512)
        self.linear_4 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.norm_1(self.linear_1(x))
        x = self.drop_1(f.relu(x))

        x = self.norm_2(self.linear_2(x))
        x = self.drop_2(f.relu(x))

        x = self.norm_3(self.linear_3(x))
        x = self.linear_4(f.relu(x))
        return {"logits": x}

## Batch Gradient Descent

In [6]:
lr = 1e-1
loss_fn = nn.CrossEntropyLoss()
model = tnn.Model(MLP())
optim = torch.optim.SGD(model.parameters(), lr=lr)

In [7]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    save_weights=False,
    device=device,
    path="../training/mnist-batch.h5",
    verbose=10,
)

In [8]:
batch_metrics = trainer.train(epochs=80)

model using cuda
training started
(epoch: 10): (train loss: 1.4270, test loss: 1.7843, train acc: 0.4883, test acc: 0.4816)
(epoch: 20): (train loss: 0.9668, test loss: 0.7176, train acc: 0.6687, test acc: 0.7613)
(epoch: 30): (train loss: 0.5820, test loss: 0.4427, train acc: 0.8017, test acc: 0.8569)
(epoch: 40): (train loss: 0.5093, test loss: 0.4371, train acc: 0.8344, test acc: 0.8577)
(epoch: 50): (train loss: 0.3980, test loss: 0.3329, train acc: 0.8807, test acc: 0.8971)
(epoch: 60): (train loss: 0.3509, test loss: 0.2790, train acc: 0.8932, test acc: 0.9156)
(epoch: 70): (train loss: 0.3067, test loss: 0.2400, train acc: 0.9085, test acc: 0.9284)
(epoch: 80): (train loss: 0.2847, test loss: 0.2219, train acc: 0.9148, test acc: 0.9339)
training complete
train_losses saved to ../training/mnist-batch.h5/metrics/train_losses
test_losses saved to ../training/mnist-batch.h5/metrics/test_losses
train_accs saved to ../training/mnist-batch.h5/metrics/train_accs
test_accs saved to ../tr

## Stochastic Gradient Descent

In [9]:
inputs = torch.tensor([ex["input"] for ex in train_dataset]).float().to(device)
labels = torch.tensor([ex["label"] for ex in train_dataset]).long().to(device)
train_tensor_dataset = data.TensorDataset(inputs, labels)

trainloader = data.DataLoader(
    train_tensor_dataset, batch_size=1, shuffle=True, drop_last=False
)

In [10]:
model = tnn.Model(MLP())
optim = torch.optim.SGD(model.parameters(), lr=lr)

In [11]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    save_weights=False,
    device=device,
    path="../training/mnist-sgd.h5",
    verbose=10,
)

In [12]:
sgd_metrics = trainer.train(epochs=80)

model using cuda
training started
(epoch: 10): (train loss: 0.0860, test loss: 0.0914, train acc: 0.9745, test acc: 0.9761)
(epoch: 20): (train loss: 0.0526, test loss: 0.0768, train acc: 0.9840, test acc: 0.9810)
(epoch: 30): (train loss: 0.0390, test loss: 0.0719, train acc: 0.9882, test acc: 0.9819)
(epoch: 40): (train loss: 0.0308, test loss: 0.0752, train acc: 0.9903, test acc: 0.9850)
(epoch: 50): (train loss: 0.0250, test loss: 0.0681, train acc: 0.9920, test acc: 0.9851)
(epoch: 60): (train loss: 0.0213, test loss: 0.0735, train acc: 0.9934, test acc: 0.9840)
(epoch: 70): (train loss: 0.0187, test loss: 0.0738, train acc: 0.9939, test acc: 0.9859)
(epoch: 80): (train loss: 0.0141, test loss: 0.0774, train acc: 0.9956, test acc: 0.9877)
training complete
train_losses saved to ../training/mnist-sgd.h5/metrics/train_losses
test_losses saved to ../training/mnist-sgd.h5/metrics/test_losses
train_accs saved to ../training/mnist-sgd.h5/metrics/train_accs
test_accs saved to ../training

## Mini-batch Gradient Descent

### Batch size 32

In [13]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=2,
)

In [14]:
model = tnn.Model(MLP())
optim = torch.optim.SGD(model.parameters(), lr=lr)

In [15]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    save_weights=False,
    device=device,
    path="../training/mnist-mini-batch-32.h5",
    verbose=10,
)

In [16]:
mini_batch_32_metrics = trainer.train(epochs=80)

model using cuda
training started
(epoch: 10): (train loss: 0.0550, test loss: 0.0626, train acc: 0.9818, test acc: 0.9820)
(epoch: 20): (train loss: 0.0318, test loss: 0.0572, train acc: 0.9891, test acc: 0.9851)
(epoch: 30): (train loss: 0.0216, test loss: 0.0524, train acc: 0.9923, test acc: 0.9873)
(epoch: 40): (train loss: 0.0169, test loss: 0.0572, train acc: 0.9943, test acc: 0.9861)
(epoch: 50): (train loss: 0.0127, test loss: 0.0596, train acc: 0.9953, test acc: 0.9866)
(epoch: 60): (train loss: 0.0105, test loss: 0.0565, train acc: 0.9965, test acc: 0.9873)
(epoch: 70): (train loss: 0.0100, test loss: 0.0698, train acc: 0.9964, test acc: 0.9849)
(epoch: 80): (train loss: 0.0079, test loss: 0.0603, train acc: 0.9971, test acc: 0.9872)
training complete
train_losses saved to ../training/mnist-mini-batch-32.h5/metrics/train_losses
test_losses saved to ../training/mnist-mini-batch-32.h5/metrics/test_losses
train_accs saved to ../training/mnist-mini-batch-32.h5/metrics/train_accs


### Batch size 64

In [17]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=2,
)

In [18]:
model = tnn.Model(MLP())
optim = torch.optim.SGD(model.parameters(), lr=lr)

In [19]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    save_weights=False,
    device=device,
    path="../training/mnist-mini-batch-64.h5",
    verbose=10,
)

In [20]:
mini_batch_64_metrics = trainer.train(epochs=80)

model using cuda
training started
(epoch: 10): (train loss: 0.0574, test loss: 0.0665, train acc: 0.9812, test acc: 0.9811)
(epoch: 20): (train loss: 0.0342, test loss: 0.0611, train acc: 0.9887, test acc: 0.9834)
(epoch: 30): (train loss: 0.0241, test loss: 0.0583, train acc: 0.9918, test acc: 0.9844)
(epoch: 40): (train loss: 0.0150, test loss: 0.0587, train acc: 0.9950, test acc: 0.9858)
(epoch: 50): (train loss: 0.0131, test loss: 0.0608, train acc: 0.9958, test acc: 0.9856)
(epoch: 60): (train loss: 0.0099, test loss: 0.0628, train acc: 0.9968, test acc: 0.9858)
(epoch: 70): (train loss: 0.0092, test loss: 0.0708, train acc: 0.9969, test acc: 0.9843)
(epoch: 80): (train loss: 0.0072, test loss: 0.0631, train acc: 0.9975, test acc: 0.9864)
training complete
train_losses saved to ../training/mnist-mini-batch-64.h5/metrics/train_losses
test_losses saved to ../training/mnist-mini-batch-64.h5/metrics/test_losses
train_accs saved to ../training/mnist-mini-batch-64.h5/metrics/train_accs


### Batch size 128

In [21]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=2,
)

In [22]:
model = tnn.Model(MLP())
optim = torch.optim.SGD(model.parameters(), lr=lr)

In [23]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    save_weights=False,
    device=device,
    path="../training/mnist-mini-batch-128.h5",
    verbose=10,
)

In [24]:
mini_batch_128_metrics = trainer.train(epochs=80)

model using cuda
training started
(epoch: 10): (train loss: 0.0620, test loss: 0.0641, train acc: 0.9800, test acc: 0.9809)
(epoch: 20): (train loss: 0.0356, test loss: 0.0562, train acc: 0.9882, test acc: 0.9835)
(epoch: 30): (train loss: 0.0229, test loss: 0.0528, train acc: 0.9921, test acc: 0.9853)
(epoch: 40): (train loss: 0.0196, test loss: 0.0599, train acc: 0.9933, test acc: 0.9852)
(epoch: 50): (train loss: 0.0143, test loss: 0.0564, train acc: 0.9951, test acc: 0.9860)
(epoch: 60): (train loss: 0.0111, test loss: 0.0604, train acc: 0.9965, test acc: 0.9863)
(epoch: 70): (train loss: 0.0096, test loss: 0.0589, train acc: 0.9968, test acc: 0.9861)
(epoch: 80): (train loss: 0.0083, test loss: 0.0588, train acc: 0.9971, test acc: 0.9862)
training complete
train_losses saved to ../training/mnist-mini-batch-128.h5/metrics/train_losses
test_losses saved to ../training/mnist-mini-batch-128.h5/metrics/test_losses
train_accs saved to ../training/mnist-mini-batch-128.h5/metrics/train_ac

### Batch size 256

In [25]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=2,
)

In [26]:
model = tnn.Model(MLP())
optim = torch.optim.SGD(model.parameters(), lr=lr)

In [27]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    save_weights=False,
    device=device,
    path="../training/mnist-mini-batch-256.h5",
    verbose=10,
)

In [28]:
mini_batch_256_metrics = trainer.train(epochs=80)

model using cuda
training started
(epoch: 10): (train loss: 0.0694, test loss: 0.0949, train acc: 0.9776, test acc: 0.9717)
(epoch: 20): (train loss: 0.0414, test loss: 0.0616, train acc: 0.9862, test acc: 0.9833)
(epoch: 30): (train loss: 0.0287, test loss: 0.0569, train acc: 0.9900, test acc: 0.9840)
(epoch: 40): (train loss: 0.0224, test loss: 0.0536, train acc: 0.9921, test acc: 0.9859)
(epoch: 50): (train loss: 0.0159, test loss: 0.0623, train acc: 0.9947, test acc: 0.9848)
(epoch: 60): (train loss: 0.0137, test loss: 0.0582, train acc: 0.9952, test acc: 0.9862)
(epoch: 70): (train loss: 0.0136, test loss: 0.0800, train acc: 0.9951, test acc: 0.9807)
(epoch: 80): (train loss: 0.0106, test loss: 0.0602, train acc: 0.9964, test acc: 0.9865)
training complete
train_losses saved to ../training/mnist-mini-batch-256.h5/metrics/train_losses
test_losses saved to ../training/mnist-mini-batch-256.h5/metrics/test_losses
train_accs saved to ../training/mnist-mini-batch-256.h5/metrics/train_ac

### Batch size 512

In [29]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=512,
    shuffle=True,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=2,
)

In [30]:
model = tnn.Model(MLP())
optim = torch.optim.SGD(model.parameters(), lr=lr)

In [31]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    save_weights=False,
    device=device,
    path="../training/mnist-mini-batch-512.h5",
    verbose=10,
)

In [32]:
mini_batch_512_metrics = trainer.train(epochs=80)

model using cuda
training started
(epoch: 10): (train loss: 0.0852, test loss: 0.0772, train acc: 0.9727, test acc: 0.9763)
(epoch: 20): (train loss: 0.0521, test loss: 0.0832, train acc: 0.9830, test acc: 0.9762)
(epoch: 30): (train loss: 0.0374, test loss: 0.0647, train acc: 0.9874, test acc: 0.9817)
(epoch: 40): (train loss: 0.0287, test loss: 0.0619, train acc: 0.9908, test acc: 0.9830)
(epoch: 50): (train loss: 0.0230, test loss: 0.0564, train acc: 0.9920, test acc: 0.9848)
(epoch: 60): (train loss: 0.0176, test loss: 0.0618, train acc: 0.9942, test acc: 0.9851)
(epoch: 70): (train loss: 0.0175, test loss: 0.0614, train acc: 0.9939, test acc: 0.9853)
(epoch: 80): (train loss: 0.0132, test loss: 0.0653, train acc: 0.9954, test acc: 0.9845)
training complete
train_losses saved to ../training/mnist-mini-batch-512.h5/metrics/train_losses
test_losses saved to ../training/mnist-mini-batch-512.h5/metrics/test_losses
train_accs saved to ../training/mnist-mini-batch-512.h5/metrics/train_ac

## SGD w/ Momentum 

### No Nesterov Accelerated Gradient

In [33]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=2,
)

In [34]:
model = tnn.Model(MLP())
optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

NameError: name 'Model' is not defined

In [ ]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    save_weights=True,
    device=device,
    path="../training/mnist-momentum-no-nag.h5",
    verbose=10,
)

In [ ]:
trainer.train(epochs=80)

### With Nesterov Accelerated Gradient

In [ ]:
model = tnn.Model(MLP())
optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True)

In [ ]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    save_weights=True,
    device=device,
    path="../training/mnist-momentum-nag.h5",
    verbose=10,
)

In [ ]:
trainer.train(epochs=80)